### Import packages

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
!pip install keras-tuner
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

ImportError: dlopen(/Users/brenda/anaconda3/lib/python3.11/site-packages/PIL/_imaging.cpython-311-darwin.so, 0x0002): Library not loaded: @rpath/libtiff.5.dylib
  Referenced from: <CA2121E1-3099-3478-89DF-9B65243DED83> /Users/brenda/anaconda3/lib/python3.11/site-packages/PIL/_imaging.cpython-311-darwin.so
  Reason: tried: '/Users/brenda/anaconda3/lib/python3.11/site-packages/PIL/../../../libtiff.5.dylib' (no such file), '/Users/brenda/anaconda3/lib/python3.11/site-packages/PIL/../../../libtiff.5.dylib' (no such file), '/Users/brenda/anaconda3/bin/../lib/libtiff.5.dylib' (no such file), '/Users/brenda/anaconda3/bin/../lib/libtiff.5.dylib' (no such file), '/usr/local/lib/libtiff.5.dylib' (no such file), '/usr/lib/libtiff.5.dylib' (no such file, not in dyld cache)

### Load and prepare data

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)
n_residual_blocks = 5

# The data, split between train and test sets
mnist = keras.datasets.mnist
(x, _), (y, _) = mnist.load_data()

# Concatenate all the images together
data = np.concatenate((x, y), axis=0)

# Round all pixel values less than 33% of the max 256 value to 0
data = np.where(data < (0.33 * 256), 0, 1)
data = data.astype(np.float32)

### Model Architecture

In [ ]:
class PixelConvLayer(layers.Layer):
    def __init__(self, mask_type, **kwargs):
        super().__init__()
        self.mask_type = mask_type
        self.conv = layers.Conv2D(**kwargs)

    def build(self, input_shape):
        self.conv.build(input_shape)
        kernel_shape = tf.shape(self.conv.kernel)
        self.mask = np.zeros(shape=kernel_shape)
        self.mask[: kernel_shape[0] // 2, ...] = 1.0
        self.mask[kernel_shape[0] // 2, : kernel_shape[1] // 2, ...] = 1.0
        if self.mask_type == "B":
            self.mask[kernel_shape[0] // 2, kernel_shape[1] // 2, ...] = 1.0

    def call(self, inputs):
        self.conv.kernel.assign(self.conv.kernel * self.mask)
        return self.conv(inputs)

class ResidualBlock(keras.layers.Layer):
    def __init__(self, filters, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = keras.layers.Conv2D(filters=filters, kernel_size=1, activation="relu")
        self.pixel_conv = PixelConvLayer(mask_type="B", filters=filters // 2, kernel_size=3, activation="relu", padding="same")
        self.conv2 = keras.layers.Conv2D(filters=filters, kernel_size=1, activation="relu")

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pixel_conv(x)
        x = self.conv2(x)
        return keras.layers.add([inputs, x])

### Hyperparameter tuning

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(PixelConvLayer(mask_type="A", filters=hp.Int('filters', 32, 128, step=32), kernel_size=7, activation="relu", padding="same", input_shape=input_shape))
    for _ in range(hp.Int('n_residual_blocks', 2, 5)):
        model.add(ResidualBlock(filters=hp.Int('filters', 32, 128, step=32)))
    model.add(layers.Conv2D(filters=1, kernel_size=1, strides=1, activation="sigmoid", padding="valid"))
    model.compile(optimizer=keras.optimizers.Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')), loss="binary_crossentropy")
    return model

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='pixelcnn_tuning',
    project_name='pixelcnn'
)

tuner.search(data, data, epochs=10, validation_split=0.1, verbose=2)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
model.fit

In [ ]:
model.fit(data, data, epochs=50, validation_split=0.1, verbose=2)

### Performance matrix

In [ ]:
# Generate a batch of images
generated_images = model.predict(data[:100])

# Flatten the images for simplicity in calculation
real_images_flat = data[:100].reshape(data[:100].shape[0], -1)
generated_images_flat = generated_images.reshape(generated_images.shape[0], -1)

# Calculate Mean Squared Error (MSE) and then take the square root for RMSE
mse = mean_squared_error(real_images_flat, generated_images_flat)
rmse = np.sqrt(mse)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(real_images_flat, generated_images_flat)

# Assuming binary cross-entropy loss, calculate it manually
epsilon = 1e-7  # to prevent log(0)
bce_loss = -np.mean(
    real_images_flat * np.log(generated_images_flat + epsilon) + 
    (1 - real_images_flat) * np.log(1 - generated_images_flat + epsilon)
)

# Compile metrics into a DataFrame
metrics_dict = {
    'Metric': ['BCE Loss', 'RMSE', 'MAE'],
    'Value': [bce_loss, rmse, mae]
}

metrics_df = pd.DataFrame(metrics_dict)
print(metrics_df)

### Display images

In [ ]:
def display_images(images, title):
    plt.figure(figsize=(10, 10))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].squeeze(), cmap='gray')
        plt.axis("off")
    plt.suptitle(title)
    plt.show()

# Display real images
display_images(data[:9], "Real Images")

# Display generated images
display_images(generated_images[:9], "Generated Images")